# Char-RNN

In [1]:
import sys
import random
import numpy as np

from itertools import chain
from hyperdash import Experiment
from contextlib import redirect_stdout, redirect_stderr

In [2]:
from keras.models import Sequential
from keras.callbacks import Callback
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence, one_hot
from keras.utils import to_categorical, print_summary, plot_model, Sequence
from keras.layers import LSTM, CuDNNLSTM, Dense, TimeDistributed, Activation, GRU

Using TensorFlow backend.


## Args

In [3]:
GPU_ACTIVE = False

## Load

In [4]:
%store -r descs
len(descs)

171161

In [5]:
random.sample(descs, 10)

['Test computation of cumulative KDE.',
 'Delete all keys in the current database',
 'Arguments to pass to test.',
 "Configure the main log according to the process\\' configuration and DCNL channel. Sets `mainlog` on self. Returns nothing.",
 'Walk though the jid dir and look for jobs',
 'test binding in a date array (with setinputsizes)',
 'Tests the _get_module method with no modules.',
 'Creates an encoded database value DCNL The result is normally formatted as "algorithm$salt$hash" and DCNL must be fewer than 128 characters.',
 'Thin wrapper for the Library Import Manager. See ImportManager for details.',
 'Creates internal structures for newly registered namespace. DCNL You can register handlers for this namespace afterwards. By default one namespace DCNL already registered (jabber:client or jabber:component:accept depending on context.']

In [6]:
MAX_SEQ_LEN = max(len(desc) for desc in descs)
MAX_SEQ_LEN

500

## Preprocessing

In [7]:
def chars_split(descs):
    chars = list(set(chain.from_iterable(desc for desc in descs)))
    char_ix = {char:ix for ix, char in enumerate(chars)}
    ix_char = {ix:char for ix, char in enumerate(chars)}
    descs = [[char_ix[char] for char in desc] for desc in descs]
    return descs, char_ix, ix_char

In [8]:
%time descs, dir_map, rev_map = chars_split(descs)
list(dir_map.items())[:5], list(rev_map.items())[:5]

CPU times: user 1.55 s, sys: 68 ms, total: 1.62 s
Wall time: 1.63 s


([('R', 0), ('j', 1), ('b', 2), (';', 3), ('v', 4)],
 [(0, 'R'), (1, 'j'), (2, 'b'), (3, ';'), (4, 'v')])

In [9]:
VOCAB_SIZE = len(dir_map)
VOCAB_SIZE

95

## TT prepare

In [10]:
BATCH_SIZE = 64

In [11]:
class TTSequence(Sequence):
    def __init__(self):
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.floor(len(descs) / BATCH_SIZE))
    
    def __getitem__(self, idx):
        batch_ids = self._ids[idx * BATCH_SIZE: (idx + 1) * BATCH_SIZE]
        seq_len = max(len(descs[di]) for di in batch_ids)
        X = np.zeros((BATCH_SIZE, seq_len, VOCAB_SIZE))
        y = np.zeros_like(X)
        
        for bi, di in enumerate(batch_ids):
            for pi, wi in enumerate(descs[di]):
                X[bi, pi, wi] = 1
            
            for pi, wi in enumerate(descs[di][1:]):
                y[bi, pi, wi] = 1
        
        return X, y
    
    def on_epoch_end(self):
        """Generate new shuffle in between epochs."""
        self._ids = np.random.permutation(len(descs))

In [12]:
tts = TTSequence()
len(tts), tts[0][0].shape, tts[0][1].shape

(2674, (64, 416, 95), (64, 416, 95))

## Model

In [13]:
HIDDEN_DIM = 100
N_LAYERS = 1
LSTM_CLASS = GRU  # LSTM if not GPU_ACTIVE else CuDNNLSTM

In [14]:
model = Sequential()
model.add(LSTM_CLASS(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), 
                     dropout=0.3, return_sequences=True))
for i in range(N_LAYERS - 1):
    model.add(LSTM_CLASS(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
print_summary(model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, None, 100)         58800     
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 95)          9595      
Total params: 68,395
Trainable params: 68,395
Non-trainable params: 0
_________________________________________________________________


In [15]:
# plot_model(model)

## Learn

In [ ]:
class HDLoss(Callback):
    def on_train_begin(self, logs={}):
        self.exp = Experiment('2.1.1: ChaRNN convergence', capture_io=False)
        
        # SUPER-hacky, but it's work
        self.exp._hd.out_buf.write = lambda _: _
    
    def on_train_end(self, logs={}):
        self.exp.end()

    def on_batch_end(self, n_batch, logs={}):
        self.exp.metric('n_batch', n_batch)
        self.exp.metric('loss', logs.get('loss'))

In [ ]:
model.fit_generator(TTSequence(), verbose=1, epochs=1,
                    callbacks=[HDLoss()],
                    use_multiprocessing=True)

Epoch 1/1
 183/2674 [=>............................] - ETA: 26:51 - loss: 0.7429

## Test

In [ ]:
...